# Language Translation

## Get the Data

In [1]:
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Text to Word Ids

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    source_id_text = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_text.split('\n')]
    target_id_text = [[target_vocab_to_int[word] for word in sentence.split()] + [target_vocab_to_int['<EOS>']]for sentence in target_text.split('\n')]
    return (source_id_text, target_id_text)

Tests Passed


### Preprocess all the data and save it

In [4]:
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point

In [5]:
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

In [6]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) == LooseVersion('1.0.0'), 'This project requires TensorFlow version 1.0  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


## Build the Neural Network

In [7]:
def model_inputs():
    inputs = tf.placeholder(tf.int32, shape=(None,None), name='input')
    targets = tf.placeholder(tf.int32, shape=(None,None), name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    return (inputs, targets, learning_rate, keep_prob)

Tests Passed


### Process Decoding Input

In [8]:
def process_decoding_input(target_data, target_vocab_to_int, batch_size):
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return dec_input

Tests Passed


### Encoding

In [9]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob):
    enc_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(rnn_size)] * num_layers)
    enc_cell = tf.contrib.rnn.DropoutWrapper(enc_cell, 1.0, keep_prob)
    _, enc_state = tf.nn.dynamic_rnn(enc_cell, rnn_inputs, dtype=tf.float32)
    return enc_state

Tests Passed


### Decoding - Training

In [10]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob):
    decoder_fn = tf.contrib.seq2seq.simple_decoder_fn_train(encoder_state)
    (train_pred, _, _) = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell,
                                                             decoder_fn,
                                                             inputs=dec_embed_input,
                                                             sequence_length=sequence_length,
                                                             scope=decoding_scope)

    return output_fn(train_pred)

Tests Passed


### Decoding - Inference

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob):
    
    infer_decoder_fn = tf.contrib.seq2seq.simple_decoder_fn_inference(output_fn, 
                                                                      encoder_state, 
                                                                      dec_embeddings, 
                                                                      start_of_sequence_id, 
                                                                      end_of_sequence_id,
                                                                      maximum_length, 
                                                                      vocab_size)
    
    inference_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell,
                                                                    infer_decoder_fn,
                                                                    scope=decoding_scope)
    
    return inference_logits

Tests Passed


### Build the Decoding Layer

In [12]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, target_vocab_to_int, keep_prob):
    
    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.DropoutWrapper(cell, 1.0, keep_prob)
    dec_cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)
    
    with tf.variable_scope("decoding") as decoding_scope:
        output_fn = lambda x: tf.contrib.layers.fully_connected(x,vocab_size,None,scope=decoding_scope)
        train_output = decoding_layer_train(encoder_state,dec_cell,dec_embed_input,sequence_length,decoding_scope,output_fn,keep_prob)
        
        start_of_sequence_id = target_vocab_to_int['<GO>']
        end_of_sequence_id = target_vocab_to_int['<EOS>']
        
    with tf.variable_scope("decoding", reuse=True) as decoding_scope:
        inf_output = decoding_layer_infer(encoder_state,
                                          dec_cell,
                                          dec_embeddings,
                                          start_of_sequence_id,
                                          end_of_sequence_id,
                                          sequence_length,
                                          vocab_size,
                                          decoding_scope,
                                          output_fn,
                                          keep_prob)
    
    return (train_output, inf_output)

Tests Passed


### Build the Neural Network

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size, rnn_size, num_layers, target_vocab_to_int):

    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, enc_embedding_size)

    encoder_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob)
    
    processed_target_data = process_decoding_input(target_data, target_vocab_to_int, batch_size)
    
    dec_embedding = tf.Variable(tf.random_uniform([target_vocab_size, dec_embedding_size]))
    dec_embed = tf.nn.embedding_lookup(dec_embedding, processed_target_data)
    
    train_output, inf_output = decoding_layer(dec_embed, 
                                              dec_embedding, 
                                              encoder_state, 
                                              target_vocab_size, 
                                              sequence_length, 
                                              rnn_size, 
                                              num_layers, 
                                              target_vocab_to_int, 
                                              keep_prob)
    
    return train_output,inf_output

Tests Passed


## Neural Network Training
### Hyperparameters

In [14]:
# Number of Epochs
epochs = 20
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 40
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 40
decoding_embedding_size = 40
# Learning Rate
learning_rate = 0.005
# Dropout Keep Probability
keep_probability = 0.8

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_source_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob = model_inputs()
    sequence_length = tf.placeholder_with_default(max_source_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)
    
    train_logits, inference_logits = seq2seq_model(
        tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(source_vocab_to_int), len(target_vocab_to_int),
        encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, target_vocab_to_int)

    tf.identity(inference_logits, 'logits')
    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            train_logits,
            targets,
            tf.ones([input_shape[0], sequence_length]))

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

### Train

In [16]:
import time

def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1]), (0,0)],
            'constant')

    return np.mean(np.equal(target, np.argmax(logits, 2)))

train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]

valid_source = helper.pad_sentence_batch(source_int_text[:batch_size])
valid_target = helper.pad_sentence_batch(target_int_text[:batch_size])

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch) in enumerate(
                helper.batch_data(train_source, train_target, batch_size)):
            start_time = time.time()
            
            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 sequence_length: target_batch.shape[1],
                 keep_prob: keep_probability})
            
            batch_train_logits = sess.run(
                inference_logits,
                {input_data: source_batch, keep_prob: 1.0})
            batch_valid_logits = sess.run(
                inference_logits,
                {input_data: valid_source, keep_prob: 1.0})
                
            train_acc = get_accuracy(target_batch, batch_train_logits)
            valid_acc = get_accuracy(np.array(valid_target), batch_valid_logits)
            end_time = time.time()
            print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.3f}, Validation Accuracy: {:>6.3f}, Loss: {:>6.3f}'
                  .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    0/538 - Train Accuracy:  0.234, Validation Accuracy:  0.316, Loss:  5.870
Epoch   0 Batch    1/538 - Train Accuracy:  0.231, Validation Accuracy:  0.316, Loss:  5.770
Epoch   0 Batch    2/538 - Train Accuracy:  0.252, Validation Accuracy:  0.316, Loss:  5.642
Epoch   0 Batch    3/538 - Train Accuracy:  0.229, Validation Accuracy:  0.316, Loss:  5.501
Epoch   0 Batch    4/538 - Train Accuracy:  0.237, Validation Accuracy:  0.316, Loss:  5.315
Epoch   0 Batch    5/538 - Train Accuracy:  0.263, Validation Accuracy:  0.316, Loss:  5.074
Epoch   0 Batch    6/538 - Train Accuracy:  0.268, Validation Accuracy:  0.316, Loss:  4.851
Epoch   0 Batch    7/538 - Train Accuracy:  0.245, Validation Accuracy:  0.316, Loss:  4.687
Epoch   0 Batch    8/538 - Train Accuracy:  0.244, Validation Accuracy:  0.316, Loss:  4.499
Epoch   0 Batch    9/538 - Train Accuracy:  0.243, Validation Accuracy:  0.316, Loss:  4.323
Epoch   0 Batch   10/538 - Train Accuracy:  0.221, Validation Accuracy

### Save Parameters

In [17]:
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [18]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence

In [19]:
def sentence_to_seq(sentence, vocab_to_int):
    
    sentence = sentence.lower()
    words = sentence.split()
    word_id_list = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in words]
    return word_id_list

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [20]:
translate_sentence = 'he saw a old yellow truck .'


translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('logits:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence], keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in np.argmax(translate_logits, 1)]))
print('  French Words: {}'.format([target_int_to_vocab[i] for i in np.argmax(translate_logits, 1)]))

Input
  Word Ids:      [129, 9, 208, 42, 116, 70, 103]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [106, 165, 148, 19, 346, 327, 271, 222, 1]
  French Words: ['il', 'a', 'vu', 'un', 'vieux', 'camion', 'rouge', '.', '<EOS>']


[Google Translate](https://translate.google.com/#en/fr/He%20saw%20an%20old%20yellow%20truck) has this French as the English: 
```
He saw an old yellow car
```
and the "correct" English to French translation as:
```
Il a vu un vieux camion jaune
```

In [21]:
translate_sentence = 'he saw an old yellow truck .'

translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('logits:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence], keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in np.argmax(translate_logits, 1)]))
print('  French Words: {}'.format([target_int_to_vocab[i] for i in np.argmax(translate_logits, 1)]))

Input
  Word Ids:      [129, 9, 2, 42, 116, 70, 103]
  English Words: ['he', 'saw', '<UNK>', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [106, 24, 293, 107, 320, 327, 84, 222, 1]
  French Words: ['il', 'aime', 'pas', 'ce', 'nouveau', 'camion', 'noir', '.', '<EOS>']


[He saw little rusty blue](https://translate.google.com/#fr/en/Il%20a%20vu%20peu%20bleu%20rouill%C3%A9)